In [29]:
import os
import requests
from IPython.display import Markdown, display, update_display

In [1]:
!pip install PyPDF2

In [3]:
import PyPDF2

In [31]:
MODEL = "deepseek-r1:1.5b"

In [33]:
!ollama pull deepseek-r1:1.5b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¸ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest 
pulling aabd4debf0c8... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 GB                         
pulling 369ca498f347... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  387 B                         
pulling 6e4c38e1172f... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 KB                         
pulling f4d24e9138dd... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  148 B                         
pulling a85fe2a2e58e... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [9]:
!pip install ollama

In [195]:
fileList = ["jsncv.pdf", "cvjeshan.pdf"]

In [417]:
class BioData:
    def __init__(self, file, text=""):
        self.file = file
        self.text = text
        with open(file, "rb") as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            num_pages = len(reader.pages)
            
            for page_num in range(num_pages):
                page = reader.pages[page_num]
                self.text += page.extract_text()

    def get_contents(self):
        return f"PDF Title:\n{self.file}\nPDF Contents:\n{self.text}\n\n"

In [430]:
system_prompt = "You are a structured data extractor. Your task is to summarize CV/resume content by strictly extracting the following details without explanations, opinions, or additional text: \
    Experience: List job titles, company names, and employment dates only (no descriptions) if you find it. \
    projects/works: List all the projects/work titles if you find it. \
    Skills: List technical/professional skills as bullet points if you find it. \
    Contact Info: Include phone number and email address if you can find it. \
    Use bold headers (Experience, Projects/Works, Skills, Contact, Certification). \
    Do not add explanations, descriptions, or additional details. Keep all sections strictly factual and brief in english language."

system_prompt += "Respond in markdown with Example format:"
system_prompt += " \
    Experience \
    [Job Title], [Company], [Dates] \
    [Job Title], [Company], [Dates] \
    [Job Title], [Company], [Dates] \
    Skills \
    [Skill 1] \
    [Skill 2] \
    Contact \
    Phone: [Number] \
    Email: [Email] \
    Certification: \
    [Certificate 1] \
    [Certificate 2] \
    Ensure no paragraphs, opinions, or extra text. "

In [432]:
def user_prompt(biodata):
    user_prompt = f"You are looking at a cv titled {biodata.file}"
    user_prompt += "Summarize the provided CV in the following format without descriptions, explanations, or additional text:"
    user_prompt += "Example output: \
    Experience \
        [Job Title], [Company Name], [Employment Dates] \
    Project/Works \
        [Title], [Description] \
    Skills \
        [Skill 1] \
        [Skill 2] \
    Contact \
        Phone: [Number] \
        Email: [Email] \
    Certification: \
        [Certificate 1] \
        [Certificate 2] \
    Strictly adhere to the format in english language."
    user_prompt += "Respond in Markdown."
    user_prompt += biodata.text
    return user_prompt

In [329]:
import ollama

def summarize(file):
    biodata = BioData(file)
    messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt(biodata)}
    ]
    response = ollama.chat(model=MODEL, messages=messages, stream=False)
    result = f"{fileList.index(biodata.file) + 1}) {biodata.file} Details: \n\n" + f"\n\n{response['message']['content']}"
    return result

In [331]:
def display_summary(file):
    summary = summarize(file)
    display(Markdown(summary))

In [469]:
for cv in fileList:
    display_summary(cv)

1) jsncv.pdf Details: 



<think>
Okay, I'm trying to figure out how to summarize the user's CV without adding any extra info or explanations. They gave a detailed example of what they want, so I need to make sure I follow that exactly.

First, I see they have an Experience section where they list job titles, company names, dates, and projects. But in their example, there are no descriptions for the experience part because it's more about details than explanations. So I should just list all the jobs with their details without adding anything extra.

Next is Projects/Works section, which lists project titles, technologies used, link, etc., but again without any additional text. In the example, each project has a description in bold. Wait, no, in the user's response they didn't bold those because they just listed them. So maybe I should format each project with its details properly.

Then there's Skills section, which lists technical skills as bullet points. The example shows them as separate items. For each skill, like JavaScript, HTML, CSS, etc., it should be a list item in bold. I need to ensure that each skill is listed without any extra explanations or text.

Contact Info is about phone number and email. Again, just include those details, no additional info.

Lastly, Certifications section lists courses with dates. In the example, they were all marked as completed. So each certification should be a bullet point under the 'Certifications' header.

I also need to make sure everything is in bold headers like Experience, Projects/Works, Skills, Contact, and Certification. The example uses markdown for bold, so I'll format each section accordingly.

Looking back at the user's input, they provided specific sections with some sample data. For instance, their objective, education details, projects, etc. I should follow the same structure but without adding any extra text beyond extracting what's listed.

I might run into issues if a project or skill isn't detailed enough, but in this case, everything seems clear and detailed enough to extract without ambiguity.

So, structuring my answer: Start with Experience section listing all jobs, company names, dates. Then Projects/Works with each project's details. Skills as bullet points for each tech skill. Contact info and certification under their respective headers.

I should double-check that I'm not missing any sections or adding extra text. Also, ensure that the formatting is correct with bold headings and proper bullets without any additional explanations.
</think>

```markdown
Experience  
[Job Title], [Company Name], [Dates]  
[Job Title], [Company Name], [Dates]  
[Job Title], [Company Name], [Dates]  

Projects/Works  
1. Portfolio Design  
Description: Portfolio Design webpage.  
Technologies: HTML, CSS, JS  
Link: https://distracted-wilson-a8d227.netlify.app  
2. Spam Detection  
Description: Detects weather the given message is a spam or not  
Technologies: Python, Jypter Notebook, gensim  
Link: https://github.com/Jikku80/SpamDetection.git  
3. Sleep Analysis  
Description: Detects weather the entity was asleep during certain time period with circumstances  
Technologies: Python, Jypter Notebook, Matplotlib, Numpy, Pandas  
Link: https://github.com/Jikku80/SleepAnalysis.git  

Skills  
JavaScript, HTML, CSS, Python, Node.js, Express.js, Git, GitHub, Postman, Azure, Naïve Bayes  

Contact  
Phone: 9860269554  
Email: jeshan.marz@gmail.com  

Certifications  
HTML CSS Fundamental Course – Sololearn, 2019  
JavaScript – Sololearn, 2018

2) cvjeshan.pdf Details: 



<think>
Okay, so I need to help someone summarize their CV into a specific format. Let me see what they provided.

First, looking at the example response, it's clear that each section has bullet points without any explanations or additional text. They used bold headers for Experience, Projects/Works, Skills, Contact, and Certification. No paragraphs, just facts in English.

The user gave a CV titled cvjeshan.pdf, which seems to have sections like Work experience with different roles and companies, academic qualifications, other skills, and contact info. I need to extract each part without any extra information or descriptions.

Starting with Experience: I'll look for job titles, company names, dates, etc., and list them under bold headers. Then Projects/Works: projects or works listed similarly. Skills as bullet points. Contact info, phone number and email. Certifications.

I should make sure not to include any extra text or explanations—just the facts in a strict format. Also, keep it factual and brief in English.

Let me go through each section of the CV:

Work Experience:
- Kolotibablo – Online 2017 – 2018: Captcha Solving
- V- infotech – Stenographer - Hattiban, Lalitpur – 2018 – 2019: Part time

I'll list the job title, company name, dates, and description.

Projects/Works:
- Captcha Solving
- Part time

Wait, but in their example response, they included 'Part time' as a separate point. I think that's acceptable unless it's a combined role with other responsibilities like projects.

Skills:
Object Oriented programming concept, design and thinking; HTML, CSS, JavaScript; C#, Node.js.

These are bullet points without descriptions.

Contact Info: Phone number and email provided.

Certifications:

Jeshan Maharjan
23/09/1998
Programming Skills: Object Oriented programming, design and thinking; HTML, CSS, JavaScript; C#, Node.js.

I'll list each certification as separate bullet points.

Now I need to make sure all these sections are in bold headers as specified. Also, the language should be English without any extra text or explanations.

Let me structure it step by step:

1. Experience:
   - Job Title 1: Company Name 1, Dates 1
   - Job Title 2: Company Name 2, Dates 2

2. Projects/Works:
   - Description 1
   - Description 2 (if applicable)

3. Skills:
   - Skill 1
   - Skill 2
   - ... (as needed)

4. Contact Info:
   - Phone: Number
   - Email: [Email]

5. Certification:
   - Cert 1
   - Cert 2

I should make sure each section is clearly separated and uses bold headers, as the example shows.

Finally, ensure that I'm only using facts from the CV, without any extra explanations or text.
</think>

**Experience**

- Job Title 1: Company Name 1, Dates 1  
- Job Title 2: Company Name 2, Dates 2  

**Projects/Works**

- Description 1  
- Description 2 (if applicable)  

**Skills**

- Skill 1  
- Skill 2  
- ...  

**Contact**

- Phone: [Number]  
- Email: [Email]  

**Certification**

- Cert 1  
- Cert 2

In [461]:
system_content = "Act as an evaluator of a tech company which is looking for a js developer your job is to systematically compare the previously provided CVs and determine the best candidate. Follow these steps: \
Comparison Criteria: \
Relevance and depth of experience to the target role/job description. \
Alignment of skills with required qualifications. \
Educational background and certifications (if provided). \
Years of experience in relevant fields. \
Technical/niche expertise (e.g., javascript, mongodb, reactjs). \
Ranking Method: \
Score each CV on a scale of 1–5 for each criterion above. \
Highlight standout strengths or weaknesses (e.g., 'CV A has 8 years of leadership experience but lacks Python skills'). \
Final Evaluation: \
Declare the top candidate with a brief justification. \
Summarize why this CV outperforms others (e.g., 'Best fit due to advanced certifications, 10+ years in fintech, and direct skill overlap'). \
Format strictly as: \
Best Candidate: [file name] \
Reason: [Reason]. \
Scores: [Scores]. \
Do not include subjective opinions, markdown, or lengthy analysis. Prioritize objectivity and brevity."

system_content += "Example Output: \
Best Candidate:  \
Reason:  \
Scores: \
CV A: Experience (0), Skills (0), Education (0)\
CV B: Experience (0), Skills (0), Education (0)"


In [463]:
user_content = "Compare the previously provided CVs and identify the best candidate based on the following criteria: \
Experience: Relevance to the target role, years in field, job titles if you find it. \
Skills: Alignment with required qualifications (technical/professional). \
Education/Certifications: If applicable. \
Technical/Niche Expertise: javascript, mongodb, node.js, or industry knowledge. \
Rank each CV numerically (1–5) per criterion and select the top candidate. \
Format your response as: \
Best Candidate: [File Name] \
Reason: [Reason]. \
Scores: [Score] \
Do not include analysis, explanations, or markdown. Keep scores and reasoning concise."
user_content += "Example Output: \
Best Candidate: CV 3 \
Reason: Reason \
Scores: CV1: Exp (0), Skills (0), Edu (0) / CV2: Exp (0), Skills (0), Edu (0) / CV3: Exp (0), Skills (0), Edu (0)"


In [ ]:
messages=[
                {"role": "system", "content": system_content},
                {"role": "user", "content": user_content}
    ]
response = ollama.chat(model=MODEL, messages=messages, stream=False)
print(response['message']['content'])